Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

IMPORT DONNEES GLOBALES (SUPPRESSION DES DOUBLONS EN MM TEMPS)

In [11]:
%%time 
df_3semaines=t.ouvrir_fichier_lapi('2019-01-28 00:00:00','2019-02-17 23:59:59')
df_3semaines=df_3semaines.drop_duplicates(['created','immat'])
df_3semaines=df_3semaines.set_index('created').sort_index()

Wall time: 1.48 s


VERIFICATION DES DONNEES

In [ ]:
#trouver les doublons
doublons=df_3semaines.reset_index()
doublons.loc[doublons.duplicated(['created','immat'],keep=False)]

In [27]:
%%time 
#verifier si des passages sont trop proches

#fonction de test d'ecart entre les passages
def ecart_passage(liste_passage, liste_camera,state) : 
    for i in range(len(liste_passage)-1):
        if (pd.to_datetime(liste_passage[i+1])-pd.to_datetime(liste_passage[i])<pd.Timedelta('00:05:00') and 
            pd.to_datetime(liste_passage[i])!=pd.to_datetime(liste_passage[i+1]) and state!='!!') : #on trouve l'enchainement en moins de 5minutes, sans prendre les doublons
            return True
    else : return False
def conserver_state(liste_state):
    if '!!' in liste_state : 
        return '!!'
    else : return liste_state[0]
def test_unicite_type(liste_l, mode='unique'):
        if mode=='unique' : 
            if len(set(liste_l))==1 :
                return liste_l[0]
            else : 
                return -1
        elif mode=='1/2' :
            if any(liste_l)==1 : 
                return 1
            else : 
                return -1

#on grouep les données et modifie les colonnes
groupe=(df_3semaines.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x),'l': lambda x : test_unicite_type(list(x),'1/2'), 
                                                              'created':lambda x: tuple(x),'state':lambda x : conserver_state(list(x)) }))
#on isole les pl
groupe_pl=groupe.loc[groupe['l']==1].copy()
#on ajoute une colonne drapeau pour localiser le pb
groupe_pl['erreur_tps_passage']=groupe_pl.apply(lambda x :  ecart_passage(x['created'], x['camera_id'], x['state']),axis=1)
#et on visualise
groupe_pl[groupe_pl['erreur_tps_passage']]

IMPORT DES DONNEES SUR UNE JOURNEE

In [350]:
df_journee=t.ouvrir_fichier_lapi('2019-01-28','2019-01-30')

In [352]:
#redresser la valeur de l
df_journee.loc[df_journee['camera_id']==15,'l']=1

LISTE DES TRAJETS

In [3]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':15, 'cam_d':11, 'trajets':[
                                                        {'cameras':[15,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':15, 'cam_d':5,'trajets':[{'cameras':[15,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5],'type_trajet':'indirect'}, 
                                                        {'cameras':[15,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':15, 'cam_d':9,'trajets':[{'cameras':[15,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[15,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':15, 'cam_d':7,'trajets':[{'cameras':[15,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A660','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A660','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A660','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A660','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A660','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A660','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [655]:
liste_complete_trajet=pd.DataFrame([[tuple(liste_cam['cameras']),value[0],value[1],value[2],value[3],len(liste_cam['cameras'])] for obj, value in liste_trajet_locale.iterrows() for liste_cam in value[4]],
             columns=['cameras','origine','destination','cam_o','cam_d','nb_cams'])
liste_complete_trajet=liste_complete_trajet.sort_values('nb_cams',ascending=False)
liste_complete_trajet

(15, 11)
(15, 11)
(12, 13)
(12, 13)
(12, 13)
(6, 13)
(15, 7)
(10, 11)
(15, 5)
(15, 5)
(8, 13)
(12, 9)
(19, 18)
(12, 13)
(12, 13)
(12, 13)
(12, 13)
(19, 18)
(12, 18)
(12, 18)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(6, 13)
(15, 7)
(10, 13)
(19, 7)
(15, 9)
(8, 13)
(8, 13)
(19, 18)
(15, 7)
(8, 9)
(10, 7)
(19, 18)
(8, 11)
(8, 18)
(12, 7)
(19, 18)
(12, 18)
(12, 18)
(6, 9)
(10, 11)
(10, 5)
(15, 5)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(19, 5)
(12, 13)
(12, 13)
(12, 13)
(12, 13)
(19, 5)
(15, 5)
(15, 5)
(6, 13)
(6, 13)
(6, 13)
(6, 18)
(10, 11)
(6, 18)
(12, 9)
(12, 9)
(12, 18)
(19, 9)
(8, 18)
(19, 18)
(6, 18)
(10, 18)
(12, 18)
(19, 7)
(6, 9)
(6, 7)
(8, 5)
(15, 11)
(12, 13)
(15, 5)
(6, 13)
(10, 11)
(12, 9)
(10, 5)
(15, 9)
(10, 13)
(8, 13)
(15, 7)
(8, 9)
(10, 7)
(8, 11)
(12, 7)
(19, 5)


IMPORTANT : VERIF DES DOUBLONS ENTRE LES TRAJETS

In [830]:
df_depart=dico_od.loc[dico_od['destination']=='A89'].copy()
jointure=df_depart.merge(dico_od, on='immat')
jointure=jointure.loc[jointure.date_cam_1_x!=jointure.date_cam_1_y].copy()
(jointure.loc[((jointure.date_cam_1_y>=jointure.date_cam_1_x) & (jointure.date_cam_1_y<=jointure.date_cam_2_x)) |
              ((jointure.date_cam_2_y>=jointure.date_cam_1_x) & (jointure.date_cam_2_y<=jointure.date_cam_2_x))])

,date_cam_1_x,immat,state_x,l_x,date_cam_2_x,cameras_x,origine_x,destination_x,o_d_x,tps_parcours_x,date_cam_1_y,state_y,l_y,date_cam_2_y,cameras_y,origine_y,destination_y,o_d_y,tps_parcours_y


CREER LE TABLEAU DE MATRICE O-D

In [814]:
#MATRICE O-D !!II!
pd.pivot_table(dico_od,values='l', index='origine', columns='destination',aggfunc='count', margins=True )

destination,A10,A62,A63,A660,A89,N10,All
origine,,,,,,,
A10,NaN,7930.0,7567.0,4457.0,693.0,NaN,20647
A62,2694.0,NaN,579.0,1459.0,1240.0,2986.0,8958
A63,7544.0,375.0,NaN,NaN,1862.0,34603.0,44384
A660,3212.0,1617.0,NaN,NaN,2370.0,8178.0,15377
A89,672.0,1793.0,2442.0,2767.0,NaN,1946.0,9620
N10,NaN,4721.0,27936.0,7842.0,1311.0,NaN,41810
All,14122.0,16436.0,38524.0,16525.0,7476.0,47713.0,140796


POURCENTAGE PL PAR PLAGE HORAIRE ET CAMERA

In [432]:
#isoler pl source
groupe_tv_type_simple=df_journee.groupby('immat')['l'].apply(lambda x:test_unicite_type(list(x),mode='1/2')).reset_index().drop_duplicates('immat')
df_pl=df_journee.join(groupe_tv_type_simple.loc[groupe_tv_type_simple['l']==1].set_index('immat'),on='immat',how='inner',rsuffix='_right')
df_pl.set_index('created',inplace=True)

#obtenir les nb de pl par heure et par camera sur la source
df_synthese_pl_tot=df_pl.groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_tot'})
df_synthese_pl_transit=dico_passag.set_index('created').groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_transit'})
df_pct_pl_transit=pd.concat([df_synthese_pl_tot,df_synthese_pl_transit], axis=1, join='inner')
df_pct_pl_transit.columns=[['nb_pl_tot','nb_pl_transit']]
df_pct_pl_transit['pct_pl_transit']=df_pct_pl_transit.apply(lambda x : float(x['nb_pl_transit'])*100 / x['nb_pl_tot'] ,axis=1)
df_pct_pl_transit

TESTS DONNEES COMPLETES

In [31]:
dico_od,  dico_passag=t.transit_temps_complet_v2('2019-01-28 00:00:00',1,liste_trajet_locale,df_3semaines)

NameError: name 'liste_trajet_locale' is not defined

In [831]:
#verifs que les destination A660
dico_od.loc[(dico_od['destination']=='A660') & (dico_od['state']!='FR')].head(25)

,date_cam_1,immat,state,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
5,2019-01-28 04:31:14,9f24c5b7cab7febaedf2d40a72946d929f84c5a6,PL,1,2019-01-28 06:23:06,"(6, 2, 18)",N10,A660,N10-A660,01:51:52
24,2019-01-28 05:31:51,0c834568c293459345a7652a8193bf77742cfc11,IT,1,2019-01-28 06:18:55,"(8, 3, 18)",A89,A660,A89-A660,00:47:04
40,2019-01-28 05:52:22,48f0f08d849466e6fefa20caae4faaed12f74de6,DE,1,2019-01-28 09:46:51,"(8, 18)",A89,A660,A89-A660,03:54:29
21,2019-01-28 06:35:01,e8838d6248bcbab26f2041560488aca2381884fd,PT,1,2019-01-28 10:23:24,"(8, 18)",A89,A660,A89-A660,03:48:23
32,2019-01-28 06:44:06,38e73bcf1b62c8de25e6d245cc5d94659400a40e,IT,1,2019-01-28 08:39:41,"(10, 18)",A62,A660,A62-A660,01:55:35
35,2019-01-28 06:46:03,ce5c0c0013756f5ec24406825a3aeb6e930e562b,BG,1,2019-01-28 07:30:10,"(10, 18)",A62,A660,A62-A660,00:44:07
7,2019-01-28 06:55:39,e404858e8b99d7ced20cbfdc0cde8eb095ca9a2a,ES,1,2019-01-28 09:41:27,"(6, 3, 18)",N10,A660,N10-A660,02:45:48
31,2019-01-28 07:59:44,2da9c14f66a652882b65149a257de9f25f5a5a24,BE,1,2019-01-28 08:19:27,"(12, 18)",A10,A660,A10-A660,00:19:43
18,2019-01-28 07:32:43,e2c5b59ff6226c300e94f4b262e11a3255b15207,BE,1,2019-01-28 08:52:34,"(8, 3, 18)",A89,A660,A89-A660,01:19:51
30,2019-01-28 07:22:55,d4f82df51dd7d8d7f75a8190cfe1df5e6ad9af57,PL,1,2019-01-28 07:45:36,"(10, 18)",A62,A660,A62-A660,00:22:41


In [30]:
# #retrouver une immatriculatiob dans les données completes
df_3semaines.loc[df_3semaines['immat']=='fffbb020a440bbec286fc41d4b70865e990b4bb7']

,camera_id,immat,fiability,l,state
created,,,,,
2019-02-10 09:38:43,4,fffbb020a440bbec286fc41d4b70865e990b4bb7,99,1,FR
2019-02-10 09:49:02,5,fffbb020a440bbec286fc41d4b70865e990b4bb7,99,1,FR
2019-02-10 09:50:56,10,fffbb020a440bbec286fc41d4b70865e990b4bb7,99,1,FR
2019-02-10 10:07:29,11,fffbb020a440bbec286fc41d4b70865e990b4bb7,99,0,FR


In [865]:
#verif sur un trajet a un horaire donnee sur un trajet global
dico_od.loc[dico_od['immat']=='2da9c14f66a652882b65149a257de9f25f5a5a24']

,date_cam_1,immat,state,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours


In [570]:
test.df_transit.o_d.unique()

array(['N10-A660', 'N10-A62', 'N10-A63', 'N10-A89', 'A10-A62', 'A10-A660',
       'A10-A63', 'A10-A89'], dtype=object)

In [68]:
#verif a un horaire donne sur un trajet simple
%time test=t.trajet(df_3semaines.loc['2019-01-29 00:00:00':'2019-01-30 15:00:00'], '2019-01-29 06:00:00', 60, [10,4])

TypeError: catching classes that do not inherit from BaseException is not allowed

In [62]:
test.tps_parcours_max_type

AttributeError: 'trajet' object has no attribute 'tps_parcours_max_type'

In [135]:
#pour test des fonctions de trajets directs
df_duree=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc['2019-01-29 21:00:00':'2019-01-29 22:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==19]
df_duree_autres_cam=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[(df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
cam1_croise_autre_cam=df_duree_cam1.reset_index().merge(df_duree_autres_cam.reset_index(), on='immat')
cam1_croise_suivant=cam1_croise_autre_cam.loc[(cam1_croise_autre_cam.loc[:,'created_x']<cam1_croise_autre_cam.loc[:,'created_y'])]
cam1_fastest_next=cam1_croise_suivant.loc[cam1_croise_suivant.groupby(['immat'])['created_y'].idxmin()]

In [ ]:
#pour test des fonctions de trajets global 
liste_trajet_od=[(15,19,4,5),(15,5),(15,4,5),(15,19,5)]
camera1, camera2=15, 5
df=df_3semaines.loc['2019-01-28 00:00:00':'2019-01-29 23:59:59']
df_duree=df.loc['2019-01-28 00:00:00':'2019-01-28 01:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==camera1]
df_duree_autres_cam=df.loc[(df.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
df_autres_cam_temp_ok=df_duree_autres_cam.loc['2019-01-29 08:00:00':'2019-01-30 08:00:00']
groupe=(df_autres_cam_temp_ok.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                        'created':lambda x: tuple(x)}))

groupe['camera_id']=groupe.apply(lambda x : filtrer_passage(x['camera_id'],df_liste_trajet),axis=1)

groupe['created']=groupe.apply(lambda x : recuperer_date_cam2(x['camera_id'],x['created'],liste_trajet_od),axis=1)

df_agrege=df_duree_cam1.join(groupe,on='immat',lsuffix='_left')[['immat', 'l','camera_id','created']].rename(columns={'created':'date_cam_2', 'camera_id':'cameras'})
df_agrege=df_agrege.reset_index().rename(columns={'created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)

df_trajet=df_agrege.loc[df_agrege['cameras'].isin(liste_trajet_od)]
df_trajet_final=(df_trajet.loc[(df_trajet['cameras'].apply(lambda x : x[-1])==camera2) & (df_trajet['l']==1)
                                  & (df_trajet['tps_parcours'] < pd.Timedelta('11:30:00'))])

In [810]:
%%time
camera1, camera2=10,18
df=df_3semaines.loc['2019-01-28 10:00:00':'2019-01-29 23:59:59']
df_duree=df.loc['2019-01-28 10:00:00':'2019-01-28 11:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==camera1]
df_duree_autres_cam=df.loc[(df.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
groupe=(df_duree_autres_cam.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                               'created':lambda x: tuple(x)}))
groupe_pl=groupe.loc[groupe['l']==1].copy()
groupe_pl['camera_id']=groupe_pl.apply(lambda x : filtrer_passage(x['camera_id'],liste_complete_trajet,camera1),axis=1)#on filtre les cameras selon la liste des trajets existants
groupe_pl['created']=groupe_pl.apply(lambda x : recuperer_date_cam2(x['camera_id'],x['created'],liste_complete_trajet,camera1),axis=1)
df_ts_trajets=(groupe_pl.reset_index().merge(liste_complete_trajet[['cameras','origine','destination']],right_on='cameras', left_on='camera_id').
               rename(columns={'created':'date_cam_2'}).drop('camera_id',axis=1))
df_ts_trajets['o_d']=df_ts_trajets.apply(lambda x : x['origine']+'-'+x['destination'],axis=1)
df_agrege=df_duree_cam1.reset_index().merge(df_ts_trajets,on='immat').drop(['camera_id', 'l_x','state','fiability'],axis=1).rename(columns={'l_y':'l','created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)
df_agrege=df_agrege.loc[df_agrege['date_cam_2'] > df_agrege['date_cam_1']]

df_joint_passag_transit=df_agrege.merge(df_duree_autres_cam.reset_index(), on='immat')
df_passag_transit1=df_joint_passag_transit.loc[(df_joint_passag_transit.apply(lambda x : x['camera_id'] in x['cameras'], axis=1))]
df_passag_transit=(df_passag_transit1.loc[df_passag_transit1.apply(lambda x : x['date_cam_1']<=x['created']<=x['date_cam_2'], axis=1)]
                [['created','camera_id','immat','fiability','l_y','state']].rename(columns={'l_y':'l'}))

Wall time: 344 ms


In [811]:
#groupe_pl.loc[groupe_pl['created']>'2019-01-28 16:18:00'].sort_values('created')
#groupe.loc['2da9c14f66a652882b65149a257de9f25f5a5a24']
df_duree_cam1.loc[df_duree_cam1['immat']=='0c48c7cb179c4a15c18f2d4bb5434a7d2611e463']

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-28 10:13:59,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,66,1,BE/SM
2019-01-28 10:33:53,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,77,1,BE/SM


In [812]:
df_agrege.loc[df_agrege['immat']=='0c48c7cb179c4a15c18f2d4bb5434a7d2611e463']

,date_cam_1,immat,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
14,2019-01-28 10:13:59,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,1,2019-01-28 10:17:38,"(10, 5)",A62,N10,A62-N10,00:03:39


In [785]:
def filtrer_passage(liste, df_liste_trajet, cam) :
    for liste_cams in [a for a in liste_complete_trajet.cameras.tolist() if a[0]==cam] :
        if liste[:len(liste_cams)]==tuple(liste_cams):
            return liste[:len(liste_cams)]
    else : return liste

In [786]:
def recuperer_date_cam2(liste,liste_created,df_liste_trajet, cam):
    for liste_cams in [a for a in liste_complete_trajet.cameras.tolist() if a[0]==cam] :
        if liste[:len(liste_cams)]==tuple(liste_cams):
            return liste_created[len(liste_cams)-1]
    else : return liste_created[-1]

In [784]:
[a for a in liste_complete_trajet.cameras.tolist() if a[0]==19]

[(19, 1, 5, 11),
 (19, 4, 5, 11),
 (19, 4, 7),
 (19, 4, 11),
 (19, 5, 11),
 (19, 1, 11),
 (19, 4, 5),
 (19, 1, 5),
 (19, 9),
 (19, 11),
 (19, 7),
 (19, 5)]

In [623]:
filtrer_passage((10, 4, 5, 3, 9, 4, 10, 5, 7, 8, 6, 3, 9),liste_complete_trajet)

(10, 4, 5)

In [760]:
for cameras in zip([15,12,8,10,19,6],range(6)):
    print (cameras)

(15, 0)
(12, 1)
(8, 2)
(10, 3)
(19, 4)
(6, 5)
